In [6]:
#from google.colab import drive
#drive.mount("/content/drive")

import datetime
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

path = input('Пожалуйста, укажите путь к директории с данными: ')

train = pd.read_excel(f"{path}/train.xlsx")
test = pd.read_excel(f"{path}/test.xlsx")
macro = pd.read_excel(f"{path}/Макропоказатели.xlsx")
metal = pd.read_excel(f"{path}/Показатели рынка металла.xlsx")
price = pd.read_excel(f"{path}/Цены на сырье.xlsx")
mater = pd.read_excel(f"{path}/Данные рынка стройматериалов.xlsx")
fuel = pd.read_excel(f"{path}/Топливо.xlsx")
lme = pd.read_excel(f"{path}/Индекс LME.xlsx")
chmf = pd.read_csv(f"{path}/CHMF Акции.csv")
magn = pd.read_csv(f"{path}/MAGN Акции.csv")
nlmk = pd.read_csv(f"{path}/NLMK Акции.csv")

!pip install catboost

lme = lme.dropna()
lme.rename(columns={'дата':'dt', 'цена':'Цена LME'}, inplace=True)

chmf = chmf[['Date', 'Price', 'Vol.']]
chmf.rename(columns = {'Date':'dt', 'Price':'Цена CHMF', 'Vol.':'Объем CHMF'}, inplace=True )
magn = magn[['Дата', 'Цена', 'Объём']]
magn.rename(columns = {'Дата':'dt', 'Цена':'Цена MAGN', 'Объём':'Объем MAGN'}, inplace=True )
nlmk = nlmk[['Date', 'Price', 'Vol.']]
nlmk.rename(columns = {'Date':'dt', 'Price':'Цена NLMK', 'Vol.':'Объем NLMK'}, inplace=True )

price_d = {'K':1,
           'К':1,
           'M':1000,
           'М':1000}

def get_volume(s):
    try:
        s = '.'.join(s.split(','))
        s = '.'.join(s.split('.')[:2])
        return float(s[:-1]) * price_d[s[-1]]
    except AttributeError:
        return s

def det_price(s):
    try:
        s = ''.join(s.split(','))
        #s = '.'.join(s.split('.')[:2])
        return float(s)
    except AttributeError:
        return s

def get_date(s):
    try:
        s = '-'.join(s.split('/'))
        s = '-'.join(s.split('.'))
        try:
            return datetime.datetime.strptime(s, '%m-%d-%Y')
        except ValueError:
            return datetime.datetime.strptime(s, '%d-%m-%Y')
    except AttributeError:
        return s


stocks = [chmf, nlmk, magn]
volume_names = ['Объем CHMF', 'Объем NLMK', 'Объем MAGN']

for i in range(len(stocks)):
    stock = stocks[i]
    volume_name = volume_names[i]

    volumes = []
    for v in stock[volume_name]:
        volumes.append(get_volume(v))

    stock[volume_name] = volumes


price_stocks = [chmf, nlmk, magn, lme]
price_names = ['Цена CHMF', 'Цена NLMK', 'Цена MAGN', 'Цена LME']

for i in range(len(price_stocks)):
    stock = price_stocks[i]
    price_name = price_names[i]

    prices = []
    for p in stock[price_name]:
        prices.append(det_price(p))

    dates = []
    for d in stock['dt']:
        dates.append(get_date(d))

    stock[price_name] = prices
    stock['dt'] = dates
    stock = stock.dropna()
    stock = stock.sort_values(by='dt')


def from_timestamp(s):
    s = str(s).split(' ')[0]
    return int(s.split('-')[0]) * 365 + int(s.split('-')[1]) * 30 + int(s.split('-')[2])

dts = list(train.dt) + list(test.dt)

days_dts = []
for d in dts:
    days_dts.append(from_timestamp(d))


def check_nan(row, data):
    for key, value in row.items():
        if value == np.nan:
            value = data[key].mean()
        row[key] = value
    return row

def fill_nan(row):
    for key, _ in row.items():
        row[key] = np.nan
    return row

dfs = [macro, metal, price, mater, fuel, lme, magn, chmf, nlmk]

c = 0
for df in dfs:
    new_df = df.iloc[:1]

    if from_timestamp(df['dt'].iloc[0]) > days_dts[0]:
        print(from_timestamp(new_df['dt'].iloc[0]), days_dts[0])
        new_df.iloc[0] = fill_nan(new_df.iloc[0])

    new_df['dt'].iloc[0] = dts[0]

    ind = 0

    if c < 5:
        for i in range(1, len(dts)):
            if days_dts[i] < from_timestamp(df['dt'].iloc[ind]):
                new_df = new_df.append(check_nan(df.iloc[ind-1], new_df))

            else:
                while days_dts[i] > from_timestamp(df['dt'].iloc[ind]):
                    ind += 1
                new_df = new_df.append(check_nan(df.iloc[ind-1], new_df))

            if from_timestamp(new_df['dt'].iloc[-1]) > days_dts[i]:
                print('leak')
                break

            new_df['dt'].iloc[-1] = dts[i]

    else:
        for i in range(1, len(dts)):
            if days_dts[i] < from_timestamp(df['dt'].iloc[ind]):
                new_df = new_df.append(df.iloc[ind-1])

            else:
                while days_dts[i] > from_timestamp(df['dt'].iloc[ind]):
                    ind += 1
                new_df = new_df.append(df.iloc[ind-1])

            if from_timestamp(new_df['dt'].iloc[-1]) > days_dts[i]:
                print('leak')
                break

            new_df['dt'].iloc[-1] = dts[i]

    train = new_df.merge(train, on=['dt'])
    test = new_df.merge(test, on=['dt'])
    c += 1


buy_volume = []

for i in range(train.shape[0]-10):
    actual_price = train['Цена на арматуру'].iloc[i]
    j = 1
    while actual_price < train['Цена на арматуру'].iloc[i+j] and j < 10:
        #print(actual_price, train['Цена на арматуру'].iloc[i+j])
        j += 1
    buy_volume.append(j)

volume_train = train.iloc[:-10]
volume_train['Объем'] = buy_volume


columns = ['dt', 'Цена на арматуру']

for key, value in volume_train.corr()['Объем'].items():
    if (value > 0.05 or value < -0.05) and volume_train[key].isnull().sum() < 10:
        #print(value, " - ", key)
        columns.append(key)

volume_train_cutted = volume_train[columns]

X_v, y_v = [], []
n = 3

for i in range(n, volume_train_cutted.shape[0]):
    batch = []
    for j in range(n):
        batch += volume_train_cutted.iloc[i-j].values.tolist()[1:-1]

    X_v.append(batch)
    y_v.append(volume_train_cutted.iloc[i]['Объем'])

X_v = np.asarray(X_v)
y_v = np.asarray(y_v)


Xv_train, Xv_test, yv_train, yv_test = train_test_split(X_v, y_v, test_size=0.2, random_state=42)


from catboost import CatBoostRegressor, CatBoostClassifier

model = CatBoostRegressor(iterations=70,
                          learning_rate=0.1,
                          depth=12,
                          loss_function="RMSE",
                          #eval_metric='AUC'
                          )

model.fit(Xv_train, yv_train, eval_set=(Xv_test, yv_test), verbose=0)


test["Объем"] = 0
testv = test[columns]
testv.drop(["Объем"], axis=1, inplace=True)

n = 3
preds = []

for i in range(testv.shape[0]):
    batch = []
    for j in range(n):
        batch += testv.iloc[max(0, i-j)].values.tolist()[1:]

    preds.append(int(model.predict(np.asarray(batch))))

test = test[['dt', 'Цена на арматуру', 'Объем']]
test['Объем'] = preds

test

Пожалуйста, укажите путь к директории с данными: /content/drive/MyDrive/ufo_hack
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


<ipython-input-6-b83eb62526b3>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lme.rename(columns={'дата':'dt', 'цена':'Цена LME'}, inplace=True)
<ipython-input-6-b83eb62526b3>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chmf.rename(columns = {'Date':'dt', 'Price':'Цена CHMF', 'Vol.':'Объем CHMF'}, inplace=True )
<ipython-input-6-b83eb62526b3>:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  magn.rename(columns = {'Дата':'dt',

735901 735510


<ipython-input-6-b83eb62526b3>:151: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['dt'].iloc[-1] = dts[i]
<ipython-input-6-b83eb62526b3>:115: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  row[key] = value
<ipython-input-6-b83eb62526b3>:140: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append(check_nan(df.iloc[ind-1], new_df))
<ipython-input-6-b83eb62526b3>:151: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pyda

735961 735510


...

738516 735510


<ipython-input-6-b83eb62526b3>:167: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['dt'].iloc[-1] = dts[i]
<ipython-input-6-b83eb62526b3>:156: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append(df.iloc[ind-1])
<ipython-input-6-b83eb62526b3>:167: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['dt'].iloc[-1] = dts[i]
<ipython-input-6-b83eb62526b3>:156: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.app

738516 735510


<ipython-input-6-b83eb62526b3>:167: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['dt'].iloc[-1] = dts[i]
<ipython-input-6-b83eb62526b3>:156: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append(df.iloc[ind-1])
<ipython-input-6-b83eb62526b3>:167: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['dt'].iloc[-1] = dts[i]
<ipython-input-6-b83eb62526b3>:156: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.app

,dt,Цена на арматуру,Объем закупки
0,2022-09-05,46000,3
1,2022-09-12,46000,3
2,2022-09-19,46000,3
3,2022-09-26,45500,3
4,2022-10-03,45500,3
5,2022-10-10,45000,3
6,2022-10-17,44000,3
7,2022-10-24,43000,3
8,2022-10-31,42000,3
9,2022-11-07,41500,3


In [8]:
test

,dt,Цена на арматуру,Объем закупки
0,2022-09-05,46000,3
1,2022-09-12,46000,3
2,2022-09-19,46000,3
3,2022-09-26,45500,3
4,2022-10-03,45500,3
5,2022-10-10,45000,3
6,2022-10-17,44000,3
7,2022-10-24,43000,3
8,2022-10-31,42000,3
9,2022-11-07,41500,3
